In [1]:
from utils import db, graficos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pylab as pl
import matplotlib.pyplot as plt
import validacao_cruzada as vl
from sklearn import svm

X, y = db.get_amostra_base(percentual=0.05)
#atributos_treinamento = db.get_atributos_treinamento()
#dados_treinamento = db.ler_base_treinamento()
#rotulos_treinamento = db.get_rotulos_treinamento()

#modelo_knn = KNeighborsClassifier()

#vld_knn = vl.validacao_5x2(modelo_knn, X, y)

#modelo_svm =  svm.SVC()

#vld_svm = vl.validacao_cruzada(modelo_svm, X, y)

#print(atributos_treinamento)

#db.ler_sample_submission()



#X_train, X_test, y_train, y_test = train_test_split(dados_treinamento, rotulos_treinamento, train_size=0.01, stratify=rotulos_treinamento)

#db.normalizar_dados(dados_treinamento)
#graficos.grafico_balanceamento_classes()

In [2]:
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, RepeatedStratifiedKFold
modelo_svm =  svm.SVC()
vld_svm = vl.validacao_5x2(modelo_svm, X, y)

#modelo_svm = svm.SVC() #set the number of neighbors according to the hyperparameter found
#scores_knn = cross_validate(modelo_svm, X, y, cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=12345),
#                        scoring=['accuracy','precision_macro', 'recall_macro', 'f1_macro'],
#                        return_train_score=True)
#for k in scores_knn.keys():
#    print(str(k) + ': Mean: ' + '{:.5f} - Standard Deviation: {:.5f}'.format(scores_knn[k].mean(),scores_knn[k].std()))

TypeError: Singleton array array(SVC(), dtype=object) cannot be considered a valid collection.

In [2]:
vld_knn

{'fit_time': array([0.02422094, 0.0202198 , 0.01333618, 0.01672316, 0.01396775,
        0.016083  , 0.01408577, 0.01308203, 0.01406789, 0.01566696]),
 'score_time': array([5.10841393, 4.13317919, 3.79217792, 3.73412895, 3.74014211,
        4.06461191, 4.43456006, 4.48006892, 4.19209886, 4.42838597]),
 'test_accuracy': array([0.96303763, 0.96354167, 0.96286962, 0.96286962, 0.96270161,
        0.96286962, 0.96303763, 0.96270161, 0.96270161, 0.9625336 ]),
 'train_accuracy': array([0.96375168, 0.96362567, 0.96383569, 0.96387769, 0.9640877 ,
        0.96396169, 0.96362567, 0.96387769, 0.96370968, 0.96396169]),
 'test_precision_macro': array([0.58183958, 0.48177083, 0.56516986, 0.56516986, 0.55326204,
        0.60691454, 0.48176164, 0.55326204, 0.55326204, 0.48175244]),
 'train_precision_macro': array([0.8055184 , 0.78188923, 0.84865717, 0.80903139, 0.87343594,
        0.84572667, 0.79436975, 0.82297838, 0.78200773, 0.83627649]),
 'test_recall_macro': array([0.50195541, 0.5       , 0.5018682

In [21]:
sample_submission = db.ler_sample_submission()
sample_submission['target']


0         0.0364
1         0.0364
2         0.0364
3         0.0364
4         0.0364
           ...  
892811    0.0364
892812    0.0364
892813    0.0364
892814    0.0364
892815    0.0364
Name: target, Length: 892816, dtype: float64

In [5]:
dados_treinamento = db.ler_base_treinamento()
dados_treinamento

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


In [18]:
dados_treinamento = pd.read_csv('bases_de_dados/kaggle_submission.csv')


892816